In [1]:
#install editable version of this repository
# !python -m pip install -e .

In [2]:
import os 
import sys
from importlib import reload

#imports 
import torch
from torch.cuda import memory_allocated, empty_cache
from datasets import load_dataset
from torch.utils.data import DataLoader

# Import local functions ==================================
import activation_extractor
# from activation_extractor import *
# reload(activation_extractor)

/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:69: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:164: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:213: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torc

In [3]:
# from activation_extractor.extractors import intermediateExtractorBase 
# reload(intermediateExtractorBase)
# from activation_extractor.inferencers import inferencerBase
# reload(inferencerBase)
from activation_extractor.model_functions import load_models, tokenize_funs, inference_funs, default_hooked_layers
reload(load_models)
reload(tokenize_funs)
reload(inference_funs)
reload(default_hooked_layers)

# from activation_extractor.model_functions import embedding_to_numpy
# reload(embedding_to_numpy)
# from activation_extractor.model_functions.embedding_to_numpy import *
reload(activation_extractor)

<module 'activation_extractor' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/__init__.py'>

# Proteins/DNA

In [14]:
model_name = "EvolutionaryScale/esm3-sm-open-v1"
inferencer = activation_extractor.Inferencer(model_name, device=None, half=False)

/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/pretrained.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/o

In [21]:
# sequences = ["AAAAA", "PPPPPP"]
inputs = {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/c2/d4c2ma_.ent']}

In [20]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(inputs) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [18]:
extractor.get_outputs()

{'encoder.sequence_embed': tensor([[[ 0.0776, -0.0176,  0.0410,  ..., -0.0295, -0.0197, -0.0006],
          [-0.0869, -0.0554,  0.0166,  ...,  0.1162,  0.1289,  0.0361],
          [-0.1182,  0.0287, -0.1475,  ...,  0.0049, -0.0176,  0.0742],
          ...,
          [-0.0869, -0.0554,  0.0166,  ...,  0.1162,  0.1289,  0.0361],
          [ 0.1367, -0.0083,  0.0413,  ...,  0.0200,  0.1426, -0.0144],
          [ 0.0598,  0.0299,  0.0173,  ..., -0.0312, -0.0217, -0.0162]]],
        grad_fn=<EmbeddingBackward0>),
 'transformer.blocks.0.attn': tensor([[[ 1.2495,  1.2279,  0.6761,  ..., -0.2669,  1.0144,  1.5246],
          [ 1.0057,  1.4087,  0.7850,  ..., -0.1678,  1.3716,  1.6205],
          [ 1.0512,  1.3525,  0.6413,  ..., -0.2086,  1.0864,  1.4830],
          ...,
          [ 1.0609,  1.3925,  0.8316,  ..., -0.1978,  1.3379,  1.6239],
          [ 1.4440,  1.3581,  0.8490,  ..., -0.2114,  1.3372,  1.5687],
          [ 1.2378,  1.2337,  0.6924,  ..., -0.2906,  0.9687,  1.5615]]],
        

# Images 

In [ ]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
model_name = "timm/vgg16.tv_in1k"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

In [ ]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(sequences) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [ ]:
extractor.get_outputs()

# Text

In [ ]:
# !pip install git+https://github.com/HazyResearch/flash-attention.git

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import requests
from datasets import load_dataset

In [4]:
ds = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")
text = ds['train']['TEXT'][0]

In [5]:
model_name = "state-spaces/mamba-1.4b-hf"
inferencer = activation_extractor.Inferencer(model_name, device="cuda", half=False)

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type, modality="sequence")
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(text) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [ ]:
seq_length=processed['attention_mask'].reshape(-1).shape[0]-1
extractor.save_outputs('test', output_id=1, 
                       emb_formats=["custom"], custom_position=seq_length)

# Multimodal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import requests
from datasets import load_dataset, Dataset

import pandas as pd

In [ ]:
ds = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")

In [ ]:
idx = 501

# Open the image file
#online
url = ds['train']['URL'][idx]
# image = Image.open(requests.get(url, stream=True).raw)
# image = np.array(image)

#locally
local_path =  "../data/train2017/"
image_path = ds['train']['URL'][idx].split("/")[-1]
# image_path = dataset['URL'][idx].split("/")[-1]
image_path = os.path.join(local_path,image_path)
img = Image.open(image_path)
img_array = np.array(img)

if len(img_array.shape)==2:
    img_array =  np.stack((img_array,) * 3, axis=-1)
    
plt.imshow(img_array)

#text
text = ds['train']['TEXT'][idx]
print(text)

#inputs
input_data = {"text":text,
             "image":img_array
             }


In [ ]:
model_name = "openai/clip-vit-base-patch32"
inferencer = activation_extractor.Inferencer(model_name, device='cpu', half=False)

In [ ]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type, modality="image-text")
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(input_data) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [ ]:
extractor.get_outputs()

# Inference Over a Dataset

## Bio

In [ ]:
import numpy as np
from activation_extractor.scripts.inference import main_inference, load_the_data

In [ ]:
model_name="Rostlab/ProstT5"
output_folder="test"
emb_format="mean"
save_method="numpy"
max_batches=1

data_args = {
        "data_type":"protein",
        "batch_size":8,
        "data_source":"huggingface",
        "dataset_name":"proteinea/remote_homology",
        "dataset_partition":"train",
        "target_col":"primary",
        # "data_source":"local",
        # "target_col":"protein_seq",
        # "input_path":"/orfeo/LTS/LADE/LT_storage/bio_data/NCBI/NCBI_GP_from_geneid_Plt1000_Glt6000.csv",
        "max_length":1000,
    }


In [ ]:
data_loader = load_the_data(**data_args)
batch = next(iter(data_loader))
batch

In [ ]:
main_inference(model_name, output_folder, emb_format, save_method, max_batches, data_args)

## For Images/text

In [4]:
import numpy as np
import pandas as pd
from PIL import Image

from activation_extractor.scripts.inference import main_inference, load_the_data

from datasets import load_dataset, Dataset

In [5]:
ds = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")
ds = pd.DataFrame(ds['train'])
ds.drop_duplicates(subset=['URL'])
ds = Dataset.from_pandas(ds)

In [6]:
model_name="EvolutionaryScale/esm3-sm-open-v1"
# model_name="google/vit-base-patch16-224-in21k"
output_folder="test"
emb_format="mean"
save_method="numpy"
max_batches=1

data_args = {
        "data_type":"protein-str",
        # "modality":"image-text",
        # "modality":"text",
        "modality":"sequence",
        "target_col":"path",
        # "modality": "image",
        "batch_size":1,
        # "data_source":"huggingface",
        # "dataset_name":"ChristophSchuhmann/MS_COCO_2017_URL_TEXT",
        # "dataset_partition":"train",
        "data_source":"local",
        "input_path":"/orfeo/LTS/LADE/LT_storage/bio_data/scope/astral-2.08-40.csv",
        # "image_source":"local",
        # "image_dir":"../data/train2017",
    }

save_args = {
        "save_method":"numpy",
        "emb_formats":['LT','FT','mean'],
        "sequence_axis": 1,
}

inference_args = {
    "sequence": False,
    "structure": True,
}

In [7]:
data_loader = load_the_data(**data_args)
input_data = next(iter(data_loader))

In [13]:
data_loader = load_the_data(**data_args)
for i, el in enumerate(data_loader):
    print(i, len(el), el)

0 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d2ux6a_.ent']}
1 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d2uxnb1.ent']}
2 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d1ux5a_.ent']}
3 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d1uxza_.ent']}
4 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d1ux6a1.ent']}
5 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d4uxaa_.ent']}
6 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d4uxua_.ent']}
7 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d3uxya_.ent']}
8 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d6uxea1.ent']}
9 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d1uxoa_.ent']}
10 1 {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/ux/d2uxqa_.ent']}
11 1 {'pdb': ['/orfeo/LTS/LADE/

In [16]:
inputs = {'pdb': ['/orfeo/LTS/LADE/LT_storage/bio_data/scope/pdbstyle-2.08/c2/d4c2mb_.ent']}

In [8]:
main_inference(model_name, output_folder, save_args, max_batches, data_args, inference_args)

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/pretrained.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/o

Output folder is: test/EvolutionaryScale/esm3-sm-open-v1


✔️ EvolutionaryScale/esm3-sm-open-v1
/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/models/vqvae.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):  # type: ignore
/orfeo/cephfs/scratch/area/evillegas/glm/esm_env/lib/python3.10/site-packages/esm/models/vqvae.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):  # type: ignore


In [ ]:
def collate_pil(batch):
    """
    Convert PIL images to numpy arrays.
    """
    batch = [np.array(img) for img in batch]

In [ ]:
ds = [ np.array(dataset["train"]["img"][i]) for i in range(10) ]

In [ ]:
ds[0].shape

In [ ]:
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms

In [ ]:
# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # Normalize the images
])

train_dataset = CIFAR100(
    root='./data',  # Directory to store the dataset
    train=True,  # Download the training dataset
    download=True,  # Download if not already downloaded
    transform=transform  # Apply transformations
)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,  # Number of samples per batch
    shuffle=True,  # Shuffle the dataset
    num_workers=2,  # Number of subprocesses to use for data loadingt
)

In [ ]:
data_loader = DataLoader(train_dataset, batch_size=2, 
                             shuffle=False, collate_fn=None)

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import numpy as np

dataset = load_dataset("uoft-cs/cifar100")
data_loader = DataLoader(dataset["train"], batch_size=2, 
                             shuffle=False, collate_fn=collate_pil)